In [1]:
import os

folder_path = '/content/drive/MyDrive/Colab Notebooks/Work/Ugrum-reka/' # путь к папке
files = os.listdir(folder_path) # список файлов в папке

for file in files:
    print(file)

t_2008.csv
t_2009.csv
t_2010.csv
t_2011.csv
t_2012.csv
t_2013.csv
t_2014.csv
t_2015.csv
t_2016.csv
t_2017.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(columns=['code_post', 'year', 'month', 'day', 'heigh', 'rainy'])

In [5]:
df

,code_post,year,month,day,heigh,rainy


In [6]:
def parse_data(row, year, post):
  if len(row[0]) > 0:
    for m in range(1,13):
      data = row[m].split(' ')
      if len(data) > 1:
        #print(1)
        df.loc[len(df)] = [post, year, m, row[0], data[0], data[1]]
      else:
        df.loc[len(df)] = [post, year, m, row[0], data[0], '']
        #print(1)
  return

In [7]:
import csv
for file in files:
  with open(f'/content/drive/MyDrive/Colab Notebooks/Work/Ugrum-reka/{file}', newline='') as csvfile:
      #print(f'читаю файл {file}')
      reader = csv.reader(csvfile, delimiter=';', quotechar='"')
      year = 0
      post = 0
      read_table_mode = False
      for row in reader:
        if read_table_mode and row[0] == 'Декада':
          read_table_mode = False
        if read_table_mode:
          parse_data(row, year, post)
        if row[0] == 'Год':
          if row[1] == '':
            year = int(row[2])
          else:
            year = int(row[1])
        if row[0] == 'Код поста':
          if row[1] == '':
            post = int(row[2])
          else:
            post = int(row[1])
        if row[0] == 'Число':
          read_table_mode = True
      #for row in reader:
          #print(', '.join(row))

In [8]:
df.sample(10)

,code_post,year,month,day,heigh,rainy
3408,9417,2008,1,6,46,_I
35463,9420,2014,4,11,208,_I
7052,9408,2009,9,30,277,
15134,9422,2010,3,22,255,_I
25585,9422,2012,2,25,282,_I
16712,9405,2011,9,29,170,
26668,9404,2013,5,22,178,
27812,9408,2013,9,24,207,
33516,9413,2014,1,4,,
10355,9518,2009,12,26,128,"""I"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52824 entries, 0 to 52823
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_post  52824 non-null  int64 
 1   year       52824 non-null  int64 
 2   month      52824 non-null  int64 
 3   day        52824 non-null  object
 4   heigh      52824 non-null  object
 5   rainy      52824 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.8+ MB


In [10]:
df['day'] = df['day'].astype(int)

In [11]:
df.shape

(52824, 6)

In [12]:
import numpy as np

In [13]:
def split_and_replace(value):
  if ":" in value or '-' in value:
    parts = value.split(":")
    new_value = parts[0]
    if '-' in new_value:
      new_value = new_value.replace("-", "")
    return new_value
  else:
    return value

df['heigh'].astype(str).apply(split_and_replace).replace("-", "") #привели к строковому типу, чтобы использовать apply
df['heigh'] = pd.to_numeric(df['heigh'], errors='coerce') # преобразовали столбец в числовой формат
df['heigh'] = df['heigh'].replace(np.nan, 0) # все наны заменили нулями

In [14]:
df.loc[df['heigh']==0, 'heigh'] = 1

In [15]:
len(df[df['heigh']==0])

0

In [16]:
len(df[df['rainy']==0])

0

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52824 entries, 0 to 52823
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_post  52824 non-null  int64  
 1   year       52824 non-null  int64  
 2   month      52824 non-null  int64  
 3   day        52824 non-null  int64  
 4   heigh      52824 non-null  float64
 5   rainy      52824 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 2.8+ MB


In [18]:
df.to_csv('data.csv', index=False)